# Milestone 1 - Introduction to Buisness Analytics

Group:

Ashish Rakesh Chandra Kukreti (s230134), Ashutosh Dhaka (s222374), Daniel Lihotský (s231868), Natsuki Sato (s231459) and Oskar Fusager (s204383)

In [58]:
import pandas as pd
import numpy as np

In [59]:
df=pd.read_csv('Trips_2018.csv')

# datetime

In [60]:
df['starttime'] = pd.to_datetime(df['starttime'])
df['stoptime'] = pd.to_datetime(df['stoptime'])

df['start_date'] = df['starttime'].dt.date
df['stop_date'] = df['stoptime'].dt.date


Cleanup

In [61]:
df = df.dropna()
df = df.drop('Unnamed: 0', axis=1)

def detect_outliers(data,feature):
    Q1=data[feature].quantile(0.25)
    Q3=data[feature].quantile(0.75)
    IQR=Q3-Q1
    outlier_threshold_low=Q1-1.5*IQR
    outlier_threshold_high=Q3+1.5*IQR
    outliers=(data[feature]<outlier_threshold_low)|(data[feature]>outlier_threshold_high)
    return outliers
 
outliers_trip_minutes=detect_outliers(df,'tripduration')
outliers_start_lat=detect_outliers(df,'start_station_latitude')
outliers_start_long=detect_outliers(df,'start_station_longitude')
outliers_end_lat=detect_outliers(df,'end_station_latitude')
outliers_end_long=detect_outliers(df,'end_station_longitude')
 
print(outliers_trip_minutes.sum())
print(outliers_start_lat.sum())
print(outliers_start_long.sum())
print(outliers_end_lat.sum())
print(outliers_end_long.sum())

all_outliers=outliers_trip_minutes|outliers_start_lat|outliers_start_long|outliers_end_lat|outliers_end_long
dfc=df[~all_outliers]


824218
10530
387467
12611
393570


In [62]:
start_coordinates=dfc[['start_station_latitude','start_station_longitude']]
 
end_coordinates=dfc[['end_station_latitude','end_station_longitude']]
 
list1=list(zip(start_coordinates["start_station_latitude"].values,start_coordinates["start_station_longitude"].values))
unique_start=set(list1)
print(len(unique_start))
 
list2=list(zip(end_coordinates["end_station_latitude"].values,end_coordinates["end_station_longitude"].values))
unique_end=set(list2)
print(len(unique_end))
 
list_coordinates=list(set(unique_start|unique_end))
len(list_coordinates)
 
intersection = list(set(list1) & set(list2))
print("Common start and end stations:",len(intersection))
start_notend = list(set(list1) - set(list2))
print("stations in start and not in end stations:",len(start_notend))
end_notstart = list(set(list2) - set(list1))
print("stations in end and not in start stations:",len(end_notstart))
 
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
data = np.array(list_coordinates)
 
scaler=StandardScaler()
clustering_data_scaled=scaler.fit_transform(data)
km1 = KMeans(n_clusters=30,random_state=42)
km1 = km1.fit(clustering_data_scaled)
clusters=km1.labels_
 
mapping={}
 
for i,j in enumerate(list_coordinates):
    mapping[j]=clusters[i]
list_start_cluster = [None] * len(start_coordinates)  # Initialize the list with None or any default value
 
for i, j in enumerate(list1):
    if j in mapping:
        list_start_cluster[i] = mapping[j]
list_end_cluster = [None] * len(end_coordinates)  # Initialize the list with None or any default value
 
for i, j in enumerate(list2):
    if j in mapping:
        list_end_cluster[i] = mapping[j]

735
735
Common start and end stations: 734
stations in start and not in end stations: 1
stations in end and not in start stations: 1


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [63]:
dfc["start_cluster_labels"]=pd.DataFrame(list_start_cluster)
dfc["end_cluster_labels"]=pd.DataFrame(list_end_cluster)
 
max_demand_cluster=dfc["start_cluster_labels"].value_counts().idxmax()

/var/folders/50/gccqklrn6191b9htzpsnct140000gn/T/ipykernel_88684/2332986753.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfc["start_cluster_labels"]=pd.DataFrame(list_start_cluster)


## Separating the data

First we filter the whole dataset to only include trips that start in the cluster with the highest demand. This is the cluster we will be working with for the prediction challenge. Next we use the trip data from January till October (included) based on starttime to define our training set. For the test set we do the same but for the remaining months of November and December and we differentiate pickups and dropoffs because we want to predict them separately.

After that we count the number of trips for each hour and each day of the training set.

You can see how the dataframe looks like below.

In [64]:
df1 = dfc[dfc['start_cluster_labels'] == max_demand_cluster] #df1 is the dataframe for the cluster with most pickups

#separating train and test data
train = df1[df1['starttime'].dt.month <= 10] 
test_pickups = df1[df1['starttime'].dt.month > 10]
test_dropoffs = df1[df1['stoptime'].dt.month > 10]

#sum the number of trips of each our of each day
train['hour'] = train['starttime'].dt.hour
train['date'] = train['starttime'].dt.date
train = train.groupby(['date', 'hour']).size().reset_index(name='counts')
train


/var/folders/50/gccqklrn6191b9htzpsnct140000gn/T/ipykernel_88684/94779663.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['hour'] = train['starttime'].dt.hour
/var/folders/50/gccqklrn6191b9htzpsnct140000gn/T/ipykernel_88684/94779663.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['date'] = train['starttime'].dt.date


,date,hour,counts
0,2018-01-01,0,9
1,2018-01-01,1,23
2,2018-01-01,2,10
3,2018-01-01,3,9
4,2018-01-01,4,3
...,...,...,...
7255,2018-10-31,19,303
7256,2018-10-31,20,224
7257,2018-10-31,21,150
7258,2018-10-31,22,123


## Training the model

First we add all of the needed f

In [65]:
from sklearn import neural_network

#add features to our train data
train['lag1'] = train['counts'].shift(1)
train['lag2'] = train['counts'].shift(2)
train['lag-1'] = train['counts'].shift(-1)
train['lag-2'] = train['counts'].shift(-2)
train['month'] = pd.to_datetime(train['date']).dt.month
train['day'] = pd.to_datetime(train['date']).dt.day
train['dayofweek'] = pd.to_datetime(train['date']).dt.dayofweek
train = train.dropna()

#sepereate features from target
x = train[['hour', 'lag1', 'lag2', 'lag-1', 'lag-2', 'month', 'day', 'dayofweek']]
y = train['counts']

#train the model
model = neural_network.MLPRegressor()
model.fit(x, y)
train

,date,hour,counts,lag1,lag2,lag-1,lag-2,month,day,dayofweek
2,2018-01-01,2,10,23.0,9.0,9.0,3.0,1,1,0
3,2018-01-01,3,9,10.0,23.0,3.0,2.0,1,1,0
4,2018-01-01,4,3,9.0,10.0,2.0,2.0,1,1,0
5,2018-01-01,5,2,3.0,9.0,2.0,5.0,1,1,0
6,2018-01-01,6,2,2.0,3.0,5.0,15.0,1,1,0
...,...,...,...,...,...,...,...,...,...,...
7253,2018-10-31,17,462,451.0,387.0,428.0,303.0,10,31,2
7254,2018-10-31,18,428,462.0,451.0,303.0,224.0,10,31,2
7255,2018-10-31,19,303,428.0,462.0,224.0,150.0,10,31,2
7256,2018-10-31,20,224,303.0,428.0,150.0,123.0,10,31,2


In [66]:
import random
from sklearn.metrics import mean_squared_error, r2_score

#pick a random day from the test set
random_day = random.choice(pd.date_range(start='2018-11-01', end='2018-12-31'))
day_to_predict = random_day + pd.DateOffset(days=1)
print('random day:', random_day.date())

#pickups and dropoffs for the day to predict grouped by hour (targets)
y_test_pickups = test_pickups[test_pickups['starttime'].dt.date == day_to_predict.date()].groupby(test_pickups['starttime'].dt.hour).size().reset_index(name='counts')
y_test_dropoffs = test_dropoffs[test_dropoffs['stoptime'].dt.date == day_to_predict.date()].groupby(test_dropoffs['stoptime'].dt.hour).size().reset_index(name='counts')

#pickups and dropoffs for the random day grouped by hour(features)
x_test_pickups = test_pickups[test_pickups['starttime'].dt.date == random_day.date()].groupby(test_pickups['starttime'].dt.hour).size().reset_index(name='counts')
x_test_dropoffs = test_dropoffs[test_dropoffs['stoptime'].dt.date == random_day.date()].groupby(test_dropoffs['stoptime'].dt.hour).size().reset_index(name='counts')

#rename columns
y_test_pickups = y_test_pickups.rename(columns={'starttime': 'hour'})
y_test_dropoffs = y_test_dropoffs.rename(columns={'stoptime': 'hour'})

x_test_pickups = x_test_pickups.rename(columns={'starttime': 'hour'})
x_test_dropoffs = x_test_dropoffs.rename(columns={'stoptime': 'hour'})

#fill in missing hours (where the count is 0)
all_hours = pd.DataFrame({'hour': range(24)})

y_test_pickups = all_hours.merge(y_test_pickups, on='hour', how='left')
y_test_pickups['counts'].fillna(0, inplace=True)
y_test_dropoffs = all_hours.merge(y_test_dropoffs, on='hour', how='left')
y_test_dropoffs['counts'].fillna(0, inplace=True)

x_test_pickups = all_hours.merge(x_test_pickups, on='hour', how='left')
x_test_pickups['counts'].fillna(0, inplace=True)
x_test_dropoffs = all_hours.merge(x_test_dropoffs, on='hour', how='left')
x_test_dropoffs['counts'].fillna(0, inplace=True)

#add features to our test set
x_test_pickups = pd.concat([x_test_pickups.tail(2), x_test_pickups, x_test_pickups.head(2)])
x_test_pickups['lag1'] = x_test_pickups['counts'].shift(1)
x_test_pickups['lag2'] = x_test_pickups['counts'].shift(2)
x_test_pickups['lag-1'] = x_test_pickups['counts'].shift(-1)
x_test_pickups['lag-2'] = x_test_pickups['counts'].shift(-2)
x_test_pickups['month'] = random_day.month
x_test_pickups['day'] = random_day.day
x_test_pickups['dayofweek'] = random_day.dayofweek
x_test_pickups = x_test_pickups.dropna()

x_test_dropoffs = pd.concat([x_test_dropoffs.tail(2), x_test_dropoffs, x_test_dropoffs.head(2)])
x_test_dropoffs['lag1'] = x_test_dropoffs['counts'].shift(1)
x_test_dropoffs['lag2'] = x_test_dropoffs['counts'].shift(2)
x_test_dropoffs['lag-1'] = x_test_dropoffs['counts'].shift(-1)
x_test_dropoffs['lag-2'] = x_test_dropoffs['counts'].shift(-2)
x_test_dropoffs['month'] = random_day.month
x_test_dropoffs['day'] = random_day.day
x_test_dropoffs['dayofweek'] = random_day.dayofweek
x_test_dropoffs = x_test_dropoffs.dropna()

#separate columns for predicting
x_test_pickups = x_test_pickups[['hour', 'lag1', 'lag2', 'lag-1', 'lag-2', 'month', 'day', 'dayofweek']]
y_test_pickups = y_test_pickups['counts'].values

x_test_dropoffs = x_test_dropoffs[['hour', 'lag1', 'lag2', 'lag-1', 'lag-2', 'month', 'day', 'dayofweek']]
y_test_dropoffs = y_test_dropoffs['counts'].values

#predict
y_pred_pickups = model.predict(x_test_pickups)
y_pred_dropoffs = model.predict(x_test_dropoffs)

#evaluate
r2_pickups = r2_score(y_test_pickups, y_pred_pickups)
r2_dropoffs = r2_score(y_test_dropoffs, y_pred_dropoffs)
mse_pickups = mean_squared_error(y_test_pickups, y_pred_pickups)
mse_dropoffs = mean_squared_error(y_test_dropoffs, y_pred_dropoffs)

#show results
res = pd.DataFrame({'Actual_Pickups': y_test_pickups, 'Predicted_Pickups': y_pred_pickups, 'Actual_Dropoffs': y_test_dropoffs, 'Predicted_Dropoffs': y_pred_dropoffs})
print('r2 score for pickups:', r2_pickups)
print('r2 score for dropoffs:', r2_dropoffs)
print('mse for pickups:', mse_pickups)
print('mse for dropoffs:', mse_dropoffs)
res


random day: 2018-11-03
r2 score for pickups: 0.909426892218552
r2 score for dropoffs: 0.923632921679419
mse for pickups: 3260.5356462051095
mse for dropoffs: 2783.901250324709


,Actual_Pickups,Predicted_Pickups,Actual_Dropoffs,Predicted_Dropoffs
0,78,65.594279,78,70.987298
1,30,29.452541,41,31.471629
2,17,14.296279,20,15.909085
3,9,2.261931,12,4.173101
4,8,3.602091,5,1.701261
5,21,6.761075,19,5.110866
6,27,30.938249,28,26.618758
7,76,83.601676,69,67.746214
8,145,154.781349,120,135.235548
9,272,210.075077,233,201.446809


# Section 3 - Exploratory Component:

Here we some different ideas (We are not sure what to chose yet):

One idea would be to examine how weather conditions, particularly precipitation, affects Citi Bike ridership in New York. By extending the dataset with weather data, we would aim to determine if there is a correlation between precipitation and the number of bike rides. This analysis will provide insights into whether rainy days discourage users from utilizing the bike-sharing system.

Another idea would be to build a decision tree model to predict bike usage likelihood based on user demographics, including gender, age, and subscription status. The model will classify users into usage categories, helping identify demographic factors that influence bike usage.

# Section 4 - Conclusions:

No conclusions yet

Progress so far (milestone1):

We showed a short introduction of the data as well as some visualization for the different features. We found some interesting inferences from the data such as the demographic and trip durations over months/hours.

We began the predictive section by performing clustering. Initially started offtrying to figure out the ideal k value using the elmow graph, but due to lack of computation power ,we could not complete it. 
We use the k value as 20 ( as specified in the project descriptions) directly and identified 20 clusters and found that cluster 1 had the highest trip duration.

Note: More work needs to be done in the exploratory section.
Data cleaning was not really required so far.
The data should also be cleaned a bit more ( remove outliers and missing values) before the predictive section is done further.
